In [40]:
import mxnet as mx
from mxnet import ndarray as nd
from mxnet import gluon
from mxnet.gluon import nn, SymbolBlock

import tvm
import nnvm

import cvm_op

import numpy as np
import math
import os
import json

In [13]:
sym_file, param_file = "/data/std_out/random_3_1/symbol", "/data/std_out/random_3_1/params"
sym_json, param_bytes = json.load(open(sym_file, "r")), open(param_file, "rb").read()
params = nnvm.compiler.load_param_dict(param_bytes)
data = np.load("/data/std_out/random_3_1/data.npy")
result = np.load("/data/std_out/random_3_1/result_0.npy")

In [241]:
name_maps = {
    'strided_slice': 'slice', 
    'conv2d': 'Convolution',
    'kernel_size': 'kernel',
    'padding': 'pad',
    'use_bias': 'no_bias',
    'strides': 'stride',
    'channels': 'num_filter',
    'dilation': 'dilate',
    'groups': 'num_group',
    
    'stride': 'step',
    
    'dense': 'FullyConnected',
    'units': 'num_hidden',
}
def mx_name(name):
    if name in name_maps:
        return name_maps[name]
    return name
def eval_attr(attr):
    new_attr = {}
    for k, v in attr.items():
        k = mx_name(k)
        if k in ['shape', 'begin', 'end', 'num_newaxis', 'axis', 'reps']:
            v = eval(v)
        elif k in ['no_bias']:
            v = not eval(v)
        new_attr[k] = v
    return new_attr

In [270]:
entry_id = sym_json['node_row_ptr']
op_attrs = [json.loads(s) for s in sym_json['attrs']['op_attrs'][1]]

out_cache = {}
for nid, op in enumerate(sym_json['nodes']):
    op_name, name = op['op'], op['name']
    inputs = op['inputs']
    attr = eval_attr(op_attrs[nid])
    if op_name == 'cvm_op':
        op_name = op['attrs']['func_name']
    print (nid, op_name, name, inputs, attr)
    if name == "data":
        out_cache[nid] = [nd.array(data).astype("float64")]
    elif op_name == "null":
        out_cache[nid] = [nd.array(params[name].asnumpy()).astype("float64")]
    
    nd_inputs = []
    for eid in inputs:
        cached_in = out_cache[eid[0]][eid[1]]
        nd_inputs.append(cached_in)
        
    if op_name in ['cvm_clip', 'cvm_left_shift', 'cvm_right_shift']:
        attr['op_type'] = op_name
        op_name = 'Custom'
    
    if op_name in ['expand_dims']:
        num_newaxis = attr['num_newaxis']
        del attr['num_newaxis']
        for _ in range(num_newaxis-1):
            nd_inputs[0] = nd.expand_dims(nd_inputs[0], **attr).astype("float64")
    if op_name in ['take'] and attr['axis'] is None:
        nd_inputs[0] = nd.array(nd_inputs[0].asnumpy().flatten())
        del attr['axis']
    if op_name in ['concatenate']:
        op_name = 'concat'
        attr['dim'] = attr['axis']
        del attr['axis']
    if op_name in ['elemwise_add']:
        nd_inputs = [n.astype("float32") for n in nd_inputs]
    if name in ['tile_4', 'tile_6']:
        reps = attr['reps'][1:]
        out = nd.tile(nd_inputs[0], reps=reps).astype("float64")
        out = nd.expand_dims(out, axis=0)
        out_cache[nid] = [nd.concat(*[out for _ in range(attr['reps'][0])], dim=0).astype("float64")]
    elif op_name != "null":
        print ([(n.shape, n.dtype) for n in nd_inputs])
        out_cache[nid] = [getattr(nd, mx_name(op_name))(*nd_inputs, **attr).astype("float64")]
        
    out = out_cache[nid][0]
    print (nid, mx_name(op_name), attr, out.shape, out.dtype)
    # if nid not in [413, 414, 415, 416, *list(range\(417, 500))]:
    # if len(out.shape) <= 5:
    if True or op_name in ['conv2d']:
        # nd.save("/data/std_out/random_3_1/nd/"+name, out)
        try:
            a = out.asnumpy()
        except:
            pass
        finally:
            if op_name in ['broadcast_add']:
                root = '/data/std_out/random_3_1/nd/'
                for i, n in enumerate(nd_inputs):
                    np.save("%s/%s_%d.in"%(root, name, i), n.asnumpy())
                np.save("%s/%s.out" % (root, name), out.asnumpy())
                open("%s/%s.attr"%(root, name), "w").write(str(attr))
                
            out = out.asnumpy().flatten()
            print (out.shape, out[:20])
            with open("/data/std_out/random_3_1/nd/"+op_name+".txt", "a+") as fout:
                fout.write(name + ": ")
                fout.write("%s %s\n" % (str(out.min()), str(out.max())))
                fout.write(" ".join([str(int(o)) for o in out[:20]]))
                fout.write("\n")
                

0 null data [] {'precision': '8', 'shape': [42, 20]}
0 null {'precision': '8', 'shape': [42, 20]} (42, 20) <class 'numpy.float64'>
(840,) [ 100. -115.  -81.    5.  -49.  -82.  -57.  -39.  109.   59.  -47.  -24.
  -93.   54.  103.   30. -118.   44.   -3.  117.]
1 reshape reshape_0 [[0, 0, 0]] {'shape': (840,)}
[((42, 20), <class 'numpy.float64'>)]
1 reshape {'shape': (840,)} (840,) <class 'numpy.float64'>
(840,) [ 100. -115.  -81.    5.  -49.  -82.  -57.  -39.  109.   59.  -47.  -24.
  -93.   54.  103.   30. -118.   44.   -3.  117.]
2 strided_slice strided_slice_0 [[1, 0, 0]] {'end': (660,), 'begin': (0,)}
[((840,), <class 'numpy.float64'>)]
2 slice {'end': (660,), 'begin': (0,)} (660,) <class 'numpy.float64'>
(660,) [ 100. -115.  -81.    5.  -49.  -82.  -57.  -39.  109.   59.  -47.  -24.
  -93.   54.  103.   30. -118.   44.   -3.  117.]
3 reshape reshape_1 [[2, 0, 0]] {'shape': [20, 33]}
[((660,), <class 'numpy.float64'>)]
3 reshape {'shape': [20, 33]} (20, 33) <class 'numpy.float64'>


(123120,) [ -54. -127. -127. -115. -127.   -2. -127. -122. -127.  -95.  -13. -127.
 -127. -127. -127.  104. -127.    6. -127. -127.]
48 strided_slice strided_slice_7 [[47, 0, 0]] {'end': (120736,), 'begin': (0,)}
[((123120,), <class 'numpy.float64'>)]
48 slice {'end': (120736,), 'begin': (0,)} (120736,) <class 'numpy.float64'>
(120736,) [ -54. -127. -127. -115. -127.   -2. -127. -122. -127.  -95.  -13. -127.
 -127. -127. -127.  104. -127.    6. -127. -127.]
49 reshape reshape_15 [[48, 0, 0]] {'shape': [56, 49, 44]}
[((120736,), <class 'numpy.float64'>)]
49 reshape {'shape': [56, 49, 44]} (56, 49, 44) <class 'numpy.float64'>
(120736,) [ -54. -127. -127. -115. -127.   -2. -127. -122. -127.  -95.  -13. -127.
 -127. -127. -127.  104. -127.    6. -127. -127.]
50 tile tile_0 [[49, 0, 0]] {'reps': [4, 2, 1, 3]}
[((56, 49, 44), <class 'numpy.float64'>)]
50 tile {'reps': [4, 2, 1, 3]} (4, 112, 49, 132) <class 'numpy.float64'>
(2897664,) [ -54. -127. -127. -115. -127.   -2. -127. -122. -127.  -9

(34787,) [-127.  -28. -114. -127.  127. -127. -127. -127.  116.  127.   80.  127.
  -42.  127.   58.  -76. -123.  127.  127.  -76.]
102 reshape reshape_28 [[101, 0, 0]] {'shape': (34787,)}
[((43, 809), <class 'numpy.float64'>)]
102 reshape {'shape': (34787,)} (34787,) <class 'numpy.float64'>
(34787,) [-127.  -28. -114. -127.  127. -127. -127. -127.  116.  127.   80.  127.
  -42.  127.   58.  -76. -123.  127.  127.  -76.]
103 strided_slice strided_slice_15 [[102, 0, 0]] {'end': (12586,), 'begin': (0,)}
[((34787,), <class 'numpy.float64'>)]
103 slice {'end': (12586,), 'begin': (0,)} (12586,) <class 'numpy.float64'>
(12586,) [-127.  -28. -114. -127.  127. -127. -127. -127.  116.  127.   80.  127.
  -42.  127.   58.  -76. -123.  127.  127.  -76.]
104 reshape reshape_29 [[103, 0, 0]] {'shape': [29, 31, 1, 14]}
[((12586,), <class 'numpy.float64'>)]
104 reshape {'shape': [29, 31, 1, 14]} (29, 31, 1, 14) <class 'numpy.float64'>
(12586,) [-127.  -28. -114. -127.  127. -127. -127. -127.  116.  1

159 strided_slice strided_slice_24 [[158, 0, 0]] {'end': (10,), 'begin': (0,)}
[((64638,), <class 'numpy.float64'>)]
159 slice {'end': (10,), 'begin': (0,)} (10,) <class 'numpy.float64'>
(10,) [  62.   18.   45. -127.  -13.   -6.   21.  116. -127. -127.]
160 reshape reshape_45 [[159, 0, 0]] {'shape': [10]}
[((10,), <class 'numpy.float64'>)]
160 reshape {'shape': [10]} (10,) <class 'numpy.float64'>
(10,) [  62.   18.   45. -127.  -13.   -6.   21.  116. -127. -127.]
161 repeat repeat_12 [[160, 0, 0]] {'axis': 0, 'repeats': '2'}
[((10,), <class 'numpy.float64'>)]
161 repeat {'axis': 0, 'repeats': '2'} (20,) <class 'numpy.float64'>
(20,) [  62.   62.   18.   18.   45.   45. -127. -127.  -13.  -13.   -6.   -6.
   21.   21.  116.  116. -127. -127. -127. -127.]
162 null parameter_25 [] {'precision': '8', 'shape': [20]}
162 null {'precision': '8', 'shape': [20]} (20,) <class 'numpy.float64'>
(20,) [  41.  -23.   60.   23.  -13.  -90. -101.   40.  -29.   54.  -15. -127.
   45.  -68. -117.  -37.

 122. 122. 122. 122. 122. 122.]
227 strided_slice strided_slice_34 [[226, 0, 0]] {'end': (684495,), 'begin': (0,)}
[((733824,), <class 'numpy.float64'>)]
227 slice {'end': (684495,), 'begin': (0,)} (684495,) <class 'numpy.float64'>
(684495,) [ 90.  90.  90.  90.  90.  90.  90. -15. -15. -15. -15. -15. -15. -15.
 122. 122. 122. 122. 122. 122.]
228 reshape reshape_63 [[227, 0, 0]] {'shape': [53, 15, 41, 21]}
[((684495,), <class 'numpy.float64'>)]
228 reshape {'shape': [53, 15, 41, 21]} (53, 15, 41, 21) <class 'numpy.float64'>
(684495,) [ 90.  90.  90.  90.  90.  90.  90. -15. -15. -15. -15. -15. -15. -15.
 122. 122. 122. 122. 122. 122.]
229 null parameter_36 [] {'precision': '8', 'shape': [53, 15, 41, 21]}
229 null {'precision': '8', 'shape': [53, 15, 41, 21]} (53, 15, 41, 21) <class 'numpy.float64'>
(684495,) [  76.  -93.    0.   65.  125.   15. -117. -102. -116.  -74. -118.   39.
  -77.  -31.   97.   33.   83.   81. -117.  -58.]
230 elemwise_add elemwise_add_26 [[228, 0, 0], [229, 0, 0

...

512 expand_dims {'axis': 1} (41, 1, 1, 1, 1, 1) <class 'numpy.float64'>
(41,) [  91.   91.   91.   91.   91.  127.  127.  127.  127.  127.   88.   88.
   88.   88.   88. -127. -127. -127. -127. -127.]
513 null parameter_78 [] {'precision': '8', 'shape': [41, 1, 1, 1, 1, 1]}
513 null {'precision': '8', 'shape': [41, 1, 1, 1, 1, 1]} (41, 1, 1, 1, 1, 1) <class 'numpy.float64'>
(41,) [  58.   75.  -18.   20.  126.  110.   66.   27.  -13.   88.  122. -116.
   54.  -95.  114.   31.  -21.   43.   57.  -90.]
514 elemwise_add elemwise_add_58 [[512, 0, 0], [513, 0, 0]] {}
[((41, 1, 1, 1, 1, 1), <class 'numpy.float32'>), ((41, 1, 1, 1, 1, 1), <class 'numpy.float32'>)]
514 elemwise_add {} (41, 1, 1, 1, 1, 1) <class 'numpy.float64'>
(41,) [ 149.  166.   73.  111.  217.  237.  193.  154.  114.  215.  210.  -28.
  142.   -7.  202.  -96. -148.  -84.  -70. -217.]
515 cvm_clip cvm_clip_67 [[514, 0, 0]] {'precision': '8'}
[((41, 1, 1, 1, 1, 1), <class 'numpy.float64'>)]
515 Custom {'precision': '8', 'op_

(1,) [139.]
556 cvm_clip cvm_clip_73 [[555, 0, 0]] {'precision': '8'}
[((1,), <class 'numpy.float64'>)]
556 Custom {'precision': '8', 'op_type': 'cvm_clip'} (1,) <class 'numpy.float64'>
(1,) [127.]
557 reshape reshape_158 [[556, 0, 0]] {'shape': (1,)}
[((1,), <class 'numpy.float64'>)]
557 reshape {'shape': (1,)} (1,) <class 'numpy.float64'>
(1,) [127.]
558 repeat repeat_39 [[557, 0, 0]] {'repeats': '11718'}
[((1,), <class 'numpy.float64'>)]
558 repeat {'repeats': '11718'} (11718,) <class 'numpy.float64'>
(11718,) [127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127.
 127. 127. 127. 127. 127. 127.]
559 strided_slice strided_slice_85 [[558, 0, 0]] {'end': (11718,), 'begin': (0,)}
[((11718,), <class 'numpy.float64'>)]
559 slice {'end': (11718,), 'begin': (0,)} (11718,) <class 'numpy.float64'>
(11718,) [127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127. 127.
 127. 127. 127. 127. 127. 127.]
560 reshape reshape_159 [[559, 0, 0]] {'shape': [7, 54, 31]}
[((11718,

605 slice {'end': (5,), 'begin': (0,)} (5,) <class 'numpy.float64'>
(5,) [ -62.  -79.  -86. -127. -127.]
606 reshape reshape_175 [[605, 0, 0]] {'shape': [1, 5]}
[((5,), <class 'numpy.float64'>)]
606 reshape {'shape': [1, 5]} (1, 5) <class 'numpy.float64'>
(5,) [ -62.  -79.  -86. -127. -127.]
607 broadcast_add broadcast_add_4 [[603, 0, 0], [606, 0, 0]] {}
[((47, 1, 1), <class 'numpy.float64'>), ((1, 5), <class 'numpy.float64'>)]
607 broadcast_add {} (47, 1, 5) <class 'numpy.float64'>
(235,) [65. 48. 41.  0.  0. 65. 48. 41.  0.  0. 65. 48. 41.  0.  0. 65. 48. 41.
  0.  0.]
608 null parameter_92 [] {'precision': '8', 'shape': [47, 1, 5]}
608 null {'precision': '8', 'shape': [47, 1, 5]} (47, 1, 5) <class 'numpy.float64'>
(235,) [-118. -104.  -25.   -2. -110.   75.  -19.   89.   33.  -14.  -32. -113.
   79.  -83.  109.   22.  -45.  -95.  -33.  -36.]
609 elemwise_add elemwise_add_69 [[607, 0, 0], [608, 0, 0]] {}
[((47, 1, 5), <class 'numpy.float32'>), ((47, 1, 5), <class 'numpy.float32'>)]
6

(13,) [ 13. 127. 127. 127.   2. 127.  33. -14.   8. -62.  29. -39.  11.]
645 reshape reshape_185 [[644, 0, 0]] {'shape': [13, 1]}
[((13,), <class 'numpy.float64'>)]
645 reshape {'shape': [13, 1]} (13, 1) <class 'numpy.float64'>
(13,) [ 13. 127. 127. 127.   2. 127.  33. -14.   8. -62.  29. -39.  11.]
646 null parameter_100 [] {'precision': '8', 'shape': [62]}
646 null {'precision': '8', 'shape': [62]} (62,) <class 'numpy.float64'>
(62,) [-116.   10.   94. -118.   71.   67.   66.  -51.  -12.  -17.  110.   80.
  111.  -57.   57.   57.   38.  100. -107.  -10.]
647 broadcast_mul broadcast_mul_5 [[645, 0, 0], [646, 0, 0]] {}
[((13, 1), <class 'numpy.float64'>), ((62,), <class 'numpy.float64'>)]
647 broadcast_mul {} (13, 62) <class 'numpy.float64'>
(806,) [-1508.   130.  1222. -1534.   923.   871.   858.  -663.  -156.  -221.
  1430.  1040.  1443.  -741.   741.   741.   494.  1300. -1391.  -130.]
648 null parameter_101 [] {'precision': '8', 'shape': [13, 62]}
648 null {'precision': '8', 'shape

(30250,) [-127. -127.  -38. -127.   89.   28. -118.    8.   81.  -84. -127. -127.
  -99. -127.  -37.  101.   58. -127.   24.  -72.]
686 strided_slice strided_slice_105 [[685, 0, 0]] {'end': (162,), 'begin': (0,)}
[((30250,), <class 'numpy.float64'>)]
686 slice {'end': (162,), 'begin': (0,)} (162,) <class 'numpy.float64'>
(162,) [-127. -127.  -38. -127.   89.   28. -118.    8.   81.  -84. -127. -127.
  -99. -127.  -37.  101.   58. -127.   24.  -72.]
687 reshape reshape_199 [[686, 0, 0]] {'shape': [6, 27]}
[((162,), <class 'numpy.float64'>)]
687 reshape {'shape': [6, 27]} (6, 27) <class 'numpy.float64'>
(162,) [-127. -127.  -38. -127.   89.   28. -118.    8.   81.  -84. -127. -127.
  -99. -127.  -37.  101.   58. -127.   24.  -72.]
688 expand_dims expand_dims_7 [[687, 0, 0]] {'num_newaxis': 4, 'axis': 0}
[((1, 1, 1, 6, 27), <class 'numpy.float64'>)]
688 expand_dims {'axis': 0} (1, 1, 1, 1, 6, 27) <class 'numpy.float64'>
(162,) [-127. -127.  -38. -127.   89.   28. -118.    8.   81.  -84. -

In [166]:
out_cache[706][0]
out_cache[706][0].asnumpy().flatten()[:20]

array([-1.76764062e-10,  4.69299033e-02,  2.70826685e+03,  1.47002199e+36,
        4.09280225e+10, -1.18496694e-01,  5.62581940e+01, -8.71756768e-13,
        1.05034179e-19,  2.79931583e+01, -9.73440977e-36, -1.07082899e-26,
        8.04505135e-07, -2.39162132e+35,  9.67414593e-29, -8.50051160e-30,
        7.63556813e+09,  7.58889252e+35, -1.11400854e-10,  1.57461577e+29],
      dtype=float32)

In [165]:
out_cache[706][0]
out_cache[706][0].asnumpy().astype('int32').flatten()[:20]

array([          0,           0,        2708, -2147483648, -2147483648,
                 0,          56,           0,           0,          27,
                 0,           0,           0, -2147483648,           0,
                 0, -2147483648, -2147483648,           0, -2147483648],
      dtype=int32)

In [126]:
result.flatten()[:20]

array([-127,    7, -127, -127,   15, -127, -127,  100,  -52,   46, -127,
        127, -121,   42,   47, -127,  127,  -79,   99,   -3], dtype=int32)

In [128]:
data.shape

(42, 20)

In [271]:
import ops_generator as opg
import utils
utils.log_init()
os.listdir("/data/std_out/random_3_1/nd")

['Custom.txt',
 'elemwise_add.txt',
 'concat.txt',
 'broadcast_add_0_1.in.npy',
 'broadcast_add_3_1.in.npy',
 'expand_dims.txt',
 'broadcast_add_4.attr',
 'tile.txt',
 'broadcast_add_2_0.in.npy',
 'broadcast_add_1.attr',
 'broadcast_add_4.out.npy',
 'broadcast_sub.txt',
 'broadcast_add_4_0.in.npy',
 'broadcast_add_3.out.npy',
 'broadcast_add_1.out.npy',
 'broadcast_add_1_0.in.npy',
 'broadcast_add_3.attr',
 'null.txt',
 'take.txt',
 'conv2d.txt',
 'squeeze.txt',
 'dense.txt',
 'broadcast_add_2_1.in.npy',
 'broadcast_add_2.attr',
 'broadcast_add_0.attr',
 'broadcast_add_3_0.in.npy',
 'repeat.txt',
 'reshape.txt',
 'strided_slice.txt',
 'transpose.txt',
 'broadcast_mul.txt',
 'broadcast_add.txt',
 'broadcast_add_0.out.npy',
 'broadcast_add_1_1.in.npy',
 'broadcast_add_0_0.in.npy',
 'broadcast_add_2.out.npy',
 'broadcast_add_4_1.in.npy']

In [275]:
opg.dump_file("broadcast_add",
            ["broadcast_add_1_0.in.npy",
             "broadcast_add_1_1.in.npy"],
            ["broadcast_add_1.out.npy"],
            "broadcast_add_1.attr",
            root="/data/std_out/random_3_1/nd")

[ 2019-06-25 01:08:35 log.ops.unittest.INFO ] Skip op:broadcast_add        hashdir=09358cb9284d4c27a0a1efa861e4e96d541c0b47


{}


In [258]:
a = np.load("/data/std_out/random_3_1/nd/tile_6_0.in.npy")
out = np.load("/data/std_out/random_3_1/nd/tile_6.out.npy")
print (a.shape, out.shape)
attr = {"reps": [3, 3, 2, 3, 2]}

(40,) (3, 3, 2, 3, 80)


In [276]:
np.product([2, 37, 55, 4])

16280